In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3

## Inserting data into categories table

In [18]:
connection=sqlite3.connect('books.db')
for category in soup.find('ul',attrs={'class':''}).find_all('a'):
    category_name=category.get_text().strip()
    sql='insert into categories (categoryName) values(?)'
    value=[category_name]
    cursor=connection.execute(sql,value)
    connection.commit()
connection.close()

## inserting data into books table

In [3]:
rate={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}
connection=sqlite3.connect('books.db')
url='https://books.toscrape.com/index.html'
response=requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')
categoryId=0
for category in soup.find('ul',attrs={'class':''}).find_all('a'):
        domain='https://books.toscrape.com/'
        url_category=str(domain+category.get('href'))
        response=requests.get(url_category)
        soup=BeautifulSoup(response.text,'html.parser')
        categoryId+=1
        while True:
            for book in soup.find_all('article',attrs={'class':'product_pod'}):
                rating=rate[book.find('p').get('class')[1]]
                book_name=book.find('h3').find('a').get('title')
                price=float(book.find('p',attrs={'class':'price_color'}).get_text()[2:])
                sql='insert into books (bookName,price,rate,categoryId) values (?,?,?,?)'
                values=[book_name,price,rating,categoryId]
                cursor=connection.execute(sql,values)
                connection.commit()
            if soup.find('li',attrs={'class':'next'}) != None:
                next_url=url_category[:url_category.rfind('/')+1]+soup.find('li',attrs={'class':'next'}).find('a').get('href')
                response=requests.get(next_url)
                soup=BeautifulSoup(response.text,'html.parser')
            else:
                break
        
connection.close()